In [1]:
# -*- encoding:utf-8 -*-
#-*- coding: cp950 -*-

import os
import sys
import itertools

import pyspark
import random

import matplotlib.pyplot as plt
import numpy as np
import numpy
from matplotlib import cm
import pandas as pd
from numpy import concatenate
from pandas import concat

from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from pyspark.sql.functions import col  
import pyspark.sql.types
from pyspark.sql.types import DoubleType
from pyspark.sql import Row
from pyspark.sql.types import Row
from pyspark.sql.functions import udf

from pyspark import since, SparkContext, keyword_only

from pyspark.ml.common import _java2py, _py2java
from pyspark.ml.wrapper import _jvm
from pyspark.ml.util import *
from pyspark.ml.wrapper import JavaEstimator, JavaModel
from pyspark.ml.param.shared import *
from pyspark.ml.common import inherit_doc

from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.feature import QuantileDiscretizer, VectorSlicer
from pyspark.ml.feature import ChiSqSelector

from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.feature import StringIndexer, OneHotEncoder,VectorAssembler
from pyspark.ml.feature import Binarizer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

from pyspark.ml.stat import Correlation, ChiSquareTest
from pyspark.ml.stat import Summarizer

from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import NGram
from pyspark.ml.feature import PCA
from pyspark.ml.feature import PolynomialExpansion
from pyspark.ml.feature import DCT
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.feature import OneHotEncoder, Normalizer, StandardScaler
from pyspark.ml.feature import MinMaxScaler, MaxAbsScaler, Bucketizer
from pyspark.ml.feature import ElementwiseProduct
from pyspark.ml.feature import SQLTransformer

from pyspark.ml.recommendation import ALS
from pyspark.ml.fpm import FPGrowth
from pyspark.ml.fpm import PrefixSpan

from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.clustering import GaussianMixture
from pyspark.ml.clustering import LDA
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.clustering import PowerIterationClustering


from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import LogisticRegression , OneVsRest
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import FMClassifier

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import AFTSurvivalRegression
from pyspark.ml.regression import IsotonicRegression
from pyspark.ml.regression import FMRegressor

from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.mllib.fpm import FPGrowth
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.classification import LogisticRegressionModel
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import StreamingLinearRegressionWithSGD



In [2]:
from pyspark.sql.session import SparkSession

# instantiate Spark
spark = SparkSession.builder.getOrCreate()

# make some test data
columns = ['customer_id', 'product_id']
vals = [
     (370, 154),
     (370, 40),
     (370, 173),
     (41, 55),
     (41, 126),
     (41, 121),
     (41, 321),
     (105, 22),
     (105, 55),
     (105, 133),
     (109, 22),
     (109, 55),
     (109, 133)    
]


# create DataFrame
df = spark.createDataFrame(vals, columns)

df.show()

+-----------+----------+
|customer_id|product_id|
+-----------+----------+
|        370|       154|
|        370|        40|
|        370|       173|
|         41|        55|
|         41|       126|
|         41|       121|
|         41|       321|
|        105|        22|
|        105|        55|
|        105|       133|
|        109|        22|
|        109|        55|
|        109|       133|
+-----------+----------+



In [3]:
# Create dataframe for FPGrowth model input
from pyspark.sql.functions import collect_list, col
from pyspark.sql import functions as F 
from pyspark.sql.functions import *
transactions = df.groupBy("customer_id")\
      .agg(F.collect_set("product_id"))

transactions.show()

+-----------+-----------------------+
|customer_id|collect_set(product_id)|
+-----------+-----------------------+
|        370|         [154, 173, 40]|
|         41|    [321, 121, 126, 55]|
|        105|          [133, 22, 55]|
|        109|          [133, 22, 55]|
+-----------+-----------------------+



In [6]:
type(transactions)

pyspark.sql.dataframe.DataFrame

In [8]:
# FPGrowth model 
from pyspark.ml.fpm import FPGrowth
mod= FPGrowth(itemsCol="collect_set(product_id)", minSupport=0.5, minConfidence=0.6)
model = mod.fit(transactions)

# Display frequent itemsets
model.freqItemsets.show()

+-------------+----+
|        items|freq|
+-------------+----+
|         [55]|   3|
|         [22]|   2|
|     [22, 55]|   2|
|        [133]|   2|
|    [133, 22]|   2|
|[133, 22, 55]|   2|
|    [133, 55]|   2|
+-------------+----+



In [9]:
# Display generated association rules.
model.associationRules.show()

+----------+----------+------------------+------------------+-------+
|antecedent|consequent|        confidence|              lift|support|
+----------+----------+------------------+------------------+-------+
| [133, 55]|      [22]|               1.0|               2.0|    0.5|
| [133, 22]|      [55]|               1.0|1.3333333333333333|    0.5|
|  [22, 55]|     [133]|               1.0|               2.0|    0.5|
|      [55]|      [22]|0.6666666666666666|1.3333333333333333|    0.5|
|      [55]|     [133]|0.6666666666666666|1.3333333333333333|    0.5|
|     [133]|      [22]|               1.0|               2.0|    0.5|
|     [133]|      [55]|               1.0|1.3333333333333333|    0.5|
|      [22]|      [55]|               1.0|1.3333333333333333|    0.5|
|      [22]|     [133]|               1.0|               2.0|    0.5|
+----------+----------+------------------+------------------+-------+



In [10]:
# transform examines the input items against all the association rules and summarise the
# consequents as prediction
model.transform(transactions).show()

+-----------+-----------------------+----------+
|customer_id|collect_set(product_id)|prediction|
+-----------+-----------------------+----------+
|        370|         [154, 173, 40]|        []|
|         41|    [321, 121, 126, 55]| [22, 133]|
|        105|          [133, 22, 55]|        []|
|        109|          [133, 22, 55]|        []|
+-----------+-----------------------+----------+

